# WCC Implementation, dataset

!pip install -q stanza 

In [ ]:
from datasets import load_dataset
import re
import pandas as pd
import json
from tqdm import tqdm
from datasets import Dataset

In [ ]:
dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)

def get_batch(dataset, batch_size):
    batch = []
    for i, example in enumerate(dataset):
        batch.append(example)
        if i + 1 == batch_size:
            break
    return batch


batch_size = 5_000_000
batch = get_batch(dataset, batch_size)

In [ ]:
stanza.download("en")
nlp = stanza.Pipeline('ru', processors='tokenize', use_gpu=True)


def split_into_sentences(paragraph):
    doc = nlp(paragraph)
    sentences = [sentence.text for sentence in doc.sentences]
    return sentences


def count_words(sentence):
    return len(sentence.split())


paragraphs = batch

data = []

for paragraph_id, paragraph in enumerate(tqdm(paragraphs), start=1):
    sentences = split_into_sentences(paragraph['text'])
    for sentence in sentences:
        word_count = count_words(sentence)
        data.append([sentence, word_count, paragraph_id])

df = pd.DataFrame(data, columns=['Sentence', 'Word Count', 'Paragraph ID'])


output_path = 'sentences.csv'
df.to_csv(output_path, index=False)